In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
from tensorflow.keras.layers import Dense

In [3]:
data=pd.read_csv("C:/Users/nchar/Desktop/english_to_french_translator/eng_-french.csv")
df=pd.DataFrame(data)
print(df.info())
df.dropna(axis=0,inplace=True)
inp=df["English words/sentences"][0:10000]
inp=list(inp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None


In [4]:
inp=[(word.split()) for word in inp]

In [5]:
inp

[['Hi.'],
 ['Run!'],
 ['Run!'],
 ['Who?'],
 ['Wow!'],
 ['Fire!'],
 ['Help!'],
 ['Jump.'],
 ['Stop!'],
 ['Stop!'],
 ['Stop!'],
 ['Wait!'],
 ['Wait!'],
 ['Go', 'on.'],
 ['Go', 'on.'],
 ['Go', 'on.'],
 ['Hello!'],
 ['Hello!'],
 ['I', 'see.'],
 ['I', 'try.'],
 ['I', 'won!'],
 ['I', 'won!'],
 ['I', 'won.'],
 ['Oh', 'no!'],
 ['Attack!'],
 ['Attack!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Get', 'up.'],
 ['Go', 'now.'],
 ['Go', 'now.'],
 ['Go', 'now.'],
 ['Got', 'it!'],
 ['Got', 'it!'],
 ['Got', 'it?'],
 ['Got', 'it?'],
 ['Got', 'it?'],
 ['Hop', 'in.'],
 ['Hop', 'in.'],
 ['Hug', 'me.'],
 ['Hug', 'me.'],
 ['I', 'fell.'],
 ['I', 'fell.'],
 ['I', 'know.'],
 ['I', 'left.'],
 ['I', 'left.'],
 ['I', 'lied.'],
 ['I', 'lost.'],
 ['I', 'paid.'],
 ["I'm", '19.'],
 ["I'm", 'OK.'],
 ["I'm", 'OK.'],
 ['Listen.'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['Really?'],
 ['Real

In [6]:
t=tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(inp)

In [7]:
inp_encds=t.texts_to_sequences(inp)
inp_encds

[[1262],
 [1263],
 [1263],
 [1264],
 [1946],
 [1265],
 [1947],
 [975],
 [976],
 [976],
 [976],
 [1266],
 [1266],
 [35, 202],
 [35, 202],
 [35, 202],
 [1267],
 [1267],
 [1, 450],
 [1, 190],
 [1, 792],
 [1, 792],
 [1, 154],
 [977, 1948],
 [1268],
 [1268],
 [793],
 [793],
 [793],
 [793],
 [33, 41],
 [35, 94],
 [35, 94],
 [35, 94],
 [46, 106],
 [46, 106],
 [46, 51],
 [46, 51],
 [46, 51],
 [1269, 100],
 [1269, 100],
 [978, 24],
 [978, 24],
 [1, 794],
 [1, 794],
 [1, 179],
 [1, 191],
 [1, 191],
 [1, 675],
 [1, 112],
 [1, 979],
 [2, 1949],
 [2, 149],
 [2, 149],
 [1270],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [795],
 [795],
 [795],
 [500],
 [7, 190],
 [7, 154],
 [7, 154],
 [7, 154],
 [7, 154],
 [123, 47],
 [1950],
 [15, 278],
 [15, 278],
 [15, 278],
 [15, 192],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 580],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [796, 12]

In [8]:
m=max([len(text) for text in inp_encds])
print(m)
inp_encds=tf.keras.preprocessing.sequence.pad_sequences(inp_encds,padding="post",maxlen=10)
#inp_encds=tf.convert_to_tensor(inp_encds)
inp_encds.shape

5


(10000, 10)

In [9]:
inp_encds

array([[1262,    0,    0, ...,    0,    0,    0],
       [1263,    0,    0, ...,    0,    0,    0],
       [1263,    0,    0, ...,    0,    0,    0],
       ...,
       [  15,   99,  163, ...,    0,    0,    0],
       [  15,   99,  163, ...,    0,    0,    0],
       [  15,   99,  163, ...,    0,    0,    0]])

In [10]:
len(t.word_index)

2946

In [12]:
embedding_dim=d_model=128
vocab_size=10000
embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
embedded_tokens=embedding_layer(inp_encds)
print(embedded_tokens.shape)

(10000, 10, 128)


In [13]:
num_heads=8
seq_len=10
dim_head=embedding_dim//num_heads
dim_head

16

In [14]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        #if mask is not None:
         #   attention_weights *=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

# Instantiate the MultiHeadAttention layer

# Example number of attention heads
mha_layer = MultiHeadAttention(d_model, num_heads)

# Call the layer
output = mha_layer(embedded_tokens)
output

# Now `output` will be the result of the multi-head attention operation


<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[ 0.0258673 , -0.04192437,  0.06004207, ...,  0.00310851,
         -0.04841687, -0.00147334],
        [ 0.0258729 , -0.04192215,  0.06004837, ...,  0.00310995,
         -0.04841808, -0.0014775 ],
        [ 0.0258729 , -0.04192215,  0.06004837, ...,  0.00310995,
         -0.04841808, -0.0014775 ],
        ...,
        [ 0.0258729 , -0.04192215,  0.06004837, ...,  0.00310995,
         -0.04841808, -0.0014775 ],
        [ 0.0258729 , -0.04192215,  0.06004837, ...,  0.00310995,
         -0.04841808, -0.0014775 ],
        [ 0.0258729 , -0.04192215,  0.06004837, ...,  0.00310995,
         -0.04841808, -0.0014775 ]],

       [[ 0.03015367, -0.03973061,  0.06404292, ...,  0.00381966,
         -0.05176181, -0.00772006],
        [ 0.03016247, -0.0397251 ,  0.06405332, ...,  0.0038238 ,
         -0.05176395, -0.00771894],
        [ 0.03016247, -0.0397251 ,  0.06405332, ...,  0.0038238 ,
         -0.05176395, -0.00771894],
        .

In [33]:
mha_layer.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_1/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_1/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_2/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_2/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_3/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_3/bias>]

In [15]:
class Feedforward(tf.keras.layers.Layer):
    def __init__(self,d_model,dff):
        super().__init__()
        self.d_model=d_model
        self.dff=dff
        self.layer1=tf.keras.layers.Dense(dff,activation="relu")
        self.layer2=tf.keras.layers.Dense(d_model)
    def call(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        return out2

In [18]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self,seq_len,d_model,num_heads,dff):
        super().__init__()
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
    def call(self,x):
        attention_out=self.attention(x)
        norm_and_add1_out=self.norm_and_add1(x+attention_out)
        feed_forward_out=self.feed_forward(norm_and_add1_out)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+feed_forward_out)
        return norm_and_add2_out

In [20]:
s=Encoder_layer(seq_len=10,d_model=128,num_heads=8,dff=1024)
out=s(embedded_tokens)
out

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-2.3477309 ,  0.6669492 , -0.16470626, ..., -0.49432355,
         -1.1234254 ,  1.1901633 ],
        [-1.2115705 ,  0.11236568, -0.53068405, ..., -0.8956122 ,
          0.2012192 ,  1.1530569 ],
        [-1.2115705 ,  0.11236568, -0.53068405, ..., -0.8956122 ,
          0.2012192 ,  1.1530569 ],
        ...,
        [-1.2115705 ,  0.11236568, -0.53068405, ..., -0.8956122 ,
          0.2012192 ,  1.1530569 ],
        [-1.2115705 ,  0.11236568, -0.53068405, ..., -0.8956122 ,
          0.2012192 ,  1.1530569 ],
        [-1.2115705 ,  0.11236568, -0.53068405, ..., -0.8956122 ,
          0.2012192 ,  1.1530569 ]],

       [[-0.26282904,  0.11766301,  0.8933668 , ..., -0.28172064,
         -1.217806  , -1.3034422 ],
        [-1.1137756 , -0.03315485, -0.5409231 , ..., -1.0255351 ,
          0.26698235,  1.1258167 ],
        [-1.1137756 , -0.03315485, -0.5409231 , ..., -1.0255351 ,
          0.26698235,  1.1258167 ],
        .

In [25]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self,seq_len,d_model,num_heads,dff,num_encoders):
        super().__init__()
        self.num_encoders=num_encoders
        self.layer=tf.keras.models.Sequential([Encoder_layer(seq_len,d_model,num_heads,dff) for _ in range(self.num_encoders)])
    def call(self,x):
        x=self.layer(x)
        return x
    

In [26]:
o=Encoder(seq_len=10,d_model=128,num_heads=8,dff=512,num_encoders=12)
d=o(embedded_tokens)
d

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.67537415, -0.5062019 ,  1.4432884 , ..., -1.1840916 ,
         -0.18954645, -0.07317699],
        [-0.72435546, -0.49114364,  1.4305062 , ..., -1.2056701 ,
         -0.18202612, -0.08132061],
        [-0.72435546, -0.49114364,  1.4305062 , ..., -1.2056701 ,
         -0.18202612, -0.08132061],
        ...,
        [-0.72435546, -0.49114364,  1.4305062 , ..., -1.2056701 ,
         -0.18202612, -0.08132061],
        [-0.72435546, -0.49114364,  1.4305062 , ..., -1.2056701 ,
         -0.18202612, -0.08132061],
        [-0.72435546, -0.49114364,  1.4305062 , ..., -1.2056701 ,
         -0.18202612, -0.08132061]],

       [[-0.5618591 , -0.5604876 ,  1.6520833 , ..., -1.2657326 ,
         -0.16805033, -0.26375118],
        [-0.5704623 , -0.53069705,  1.6549573 , ..., -1.2680792 ,
         -0.18939483, -0.25354445],
        [-0.5704623 , -0.53069705,  1.6549573 , ..., -1.2680792 ,
         -0.18939483, -0.25354445],
        .

In [32]:
o.layer.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ encoder_layer_27 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_28 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_29 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_30 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_31 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_32 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_33 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_34 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_35 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_36 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_37 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_layer_38 (Encoder_layer)     │ (10000, 10, 128)            │         198,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,379,264 (9.08 MB)

 Trainable params: 2,379,264 (9.08 MB)

 Non-trainable params: 0 (0.00 B)

"THERE ARE ANOTHER TYPES TO IMPLEMENT THE ENCODER THEY ARE":


self.layers = tf.keras.layers.LayerList([ Encoder_layer(seq_len, d_model, num_heads, dff) for _ in range(self.num_encoders) ])
def call(self, x, training=False):
    for layer in self.layers:
        x = layer(x, training=training)
    return x

and  another one will be:

self.layers=[ Encoder_layer(seq_len, d_model, num_heads, dff) for _ in range(self.num_encoders) ]
def call(self, x, training=False):
        for i in range(self.num_encoders) :
            x=self.layers[i](x)
        return x